In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [4]:
import torch
import torch.nn as nn
import vision.utils as utils
import torchvision
from alg.senn_conditional import ConditionalLinearWrapper

ortho = utils.loadOTSModel(torchvision.models.resnet18, num_classes=10, pretrained=False)
utils.prep_resnet_for_maml(ortho)
basic_block_predicate = lambda m: isinstance(m, torchvision.models.resnet.BasicBlock)
n_hidden = lambda m: m.conv2.weight.shape[0]
ConditionalLinearWrapper.wrap_model(ortho, n_hidden, -3, basic_block_predicate)
ortho.load_state_dict(torch.load('/iris/u/clin/code/editable_nlp/src/outputs/2021-05-18/21-40-59/models/finetune_epoch99_ts9999.2021-05-18.21-40-59'))


Wrapped 8 modules using predicate:
basic_block_predicate = lambda m: isinstance(m, torchvision.models.resnet.BasicBlock)

Overriding existing `inner_params` implementation
n default inner params: 17
n inner params: 15


<All keys matched successfully>

In [10]:
for p in ortho.phi():
    mat = p.data
    I = torch.eye(mat.shape[0], device=mat.device)
    A = mat.triu(1) - mat.triu(1).permute(-1,-2)
    weight = (I + A) @ (I - A).inverse()
    diff = (weight - torch.eye(weight.shape[0])).norm().item()
    print(weight.shape, diff)
    # print(p.data.norm().item())



torch.Size([64, 64]) 0.8430989980697632
torch.Size([64, 64]) 0.8345901966094971
torch.Size([128, 128]) 2.041126012802124
torch.Size([128, 128]) 3.076390027999878
torch.Size([256, 256]) 5.9811248779296875
torch.Size([256, 256]) 5.479241847991943
torch.Size([512, 512]) 10.173482894897461
torch.Size([512, 512]) 6.28060245513916


In [32]:
senn_50 = utils.loadOTSModel(torchvision.models.resnet18, num_classes=10, pretrained=False)
utils.prep_resnet_for_maml(senn_50)
basic_block_predicate = lambda m: isinstance(m, torchvision.models.resnet.BasicBlock)
n_hidden = lambda m: m.conv2.weight.shape[0]
ConditionalLinearWrapper.wrap_model(senn_50, n_hidden, -3, basic_block_predicate)
senn_50.load_state_dict(torch.load('/iris/u/clin/code/editable_nlp/src/outputs/2021-05-18/15-28-22/models/finetune_epoch49_ts9999.2021-05-18.15-28-22'))



Wrapped 8 modules for predicate <function <lambda> at 0x7fee2c510a60>
Overriding existing `inner_params` implementation
n default inner params: 17
n inner params: 15


<All keys matched successfully>

In [36]:
for p in senn_50.phi():
#     mat = p.data
#     diff = ((mat.T @ mat) - torch.eye(mat.shape[0])).norm().item()
#     print(diff)
    print(p.data.norm().item())



8.047264099121094
8.064950942993164
11.475232124328613
11.526585578918457
16.387601852416992
16.389881134033203
23.38377571105957
23.34132957458496


In [11]:
senn = utils.loadOTSModel(torchvision.models.resnet18, num_classes=10, pretrained=False)
utils.prep_resnet_for_maml(senn)
basic_block_predicate = lambda m: isinstance(m, torchvision.models.resnet.BasicBlock)
n_hidden = lambda m: m.conv2.weight.shape[0]
ConditionalLinearWrapper.wrap_model(senn, n_hidden, -3, basic_block_predicate)
senn.load_state_dict(torch.load('/iris/u/clin/code/editable_nlp/src/outputs/2021-05-18/15-28-22/models/finetune_epoch49_ts9999.2021-05-18.15-28-22'))

Wrapped 8 modules using predicate:
basic_block_predicate = lambda m: isinstance(m, torchvision.models.resnet.BasicBlock)

Overriding existing `inner_params` implementation
n default inner params: 17
n inner params: 15


<All keys matched successfully>

In [19]:
for p in senn.phi():
    weight = p.data
    U, S, Vh = torch.linalg.svd(weight)
    print(S.shape, S.max(), S.min())
    

torch.Size([64]) tensor(1.1388) tensor(0.8865)
torch.Size([64]) tensor(1.1391) tensor(0.8797)
torch.Size([128]) tensor(1.2611) tensor(0.7273)
torch.Size([128]) tensor(1.2772) tensor(0.0977)
torch.Size([256]) tensor(1.6191) tensor(0.2256)
torch.Size([256]) tensor(1.6301) tensor(0.5316)
torch.Size([512]) tensor(2.2300) tensor(0.0439)
torch.Size([512]) tensor(2.7057) tensor(0.6069)
